# Core Analysis - Morphology and Porosity

This notebook demonstrates comprehensive morphological and porosity analysis of XCT data. Learn how to:

- **Estimate filament diameters** using different methods
- **Analyze channel widths** for flow characterization
- **Analyze porosity distribution** along printing direction
- **Perform slice analysis** along and perpendicular to flow
- **Fit statistical distributions** to morphological parameters
- **Visualize results** interactively

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
1. Estimate filament diameters and channel widths
2. Analyze porosity distribution and gradients
3. Perform slice-based analysis
4. Fit distributions to morphological parameters
5. Interpret results for thermomagnetic element design

## ⚠️ Prerequisites

- **Notebook 01**: Basic understanding of loading and segmenting volumes
- **Notebook 02**: Understanding of preprocessing and filtering
- **Required packages**: Same as previous notebooks
- **Segmented volume**: Binary segmented volume ready for analysis

## 📖 Usage

1. Run all cells to initialize the widgets
2. Load a segmented volume
3. Configure analysis parameters (directions, methods)
4. Run filament, channel, and porosity analysis
5. Explore slice analysis
6. Fit distributions and visualize results


## 1. Setup and Imports


In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
from typing import Dict, List, Optional, Tuple, Any

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check for ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output, Tab, interactive
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("❌ ipywidgets not available!")
    print("   Install with: pip install ipywidgets")

# Find project root
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'src').exists():
    project_root = current_dir
else:
    project_root = current_dir

# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print("📦 Core Analysis - Morphology and Porosity")
print(f"   Project root: {project_root}")
print(f"   Widgets available: {WIDGETS_AVAILABLE}")


📦 Core Analysis - Morphology and Porosity
   Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/XCT_Thermomagnetic_Analysis
   Widgets available: True


## 2. Load Framework Modules


In [ ]:
# Load core analysis modules
try:
    from src.analyzer import XCTAnalyzer
    from src.core.filament_analysis import estimate_filament_diameter, estimate_channel_width
    from src.core.porosity import (
        analyze_porosity_distribution, porosity_along_direction,
        pore_size_distribution
    )
    from src.core.slice_analysis import (
        analyze_slice_along_flow, analyze_slice_perpendicular_flow
    )
    from src.core.visualization import (
        visualize_slice, plot_porosity_profile, plot_metrics_comparison
    )
    from src.preprocessing.statistics import (
        fit_gaussian, fit_poisson, compare_fits, evaluate_fit_quality
    )
    from src.utils.utils import load_volume, normalize_path
    
    print("✅ All modules loaded successfully")
except ImportError as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise


✅ All modules loaded successfully


## 3. Interactive Analysis Dashboard

Use the interactive widgets below to perform comprehensive morphological and porosity analysis.


In [3]:
if not WIDGETS_AVAILABLE:
    print("❌ Cannot create widgets - ipywidgets not available")
else:
    print("🎨 Creating interactive widgets...")
    
    # Initialize state
    analyzer = None
    current_volume = None
    analysis_results = {}
    
    # ============================================
    # Section 1: Data Loading
    # ============================================
    
    file_path_text = widgets.Text(
        value='',
        placeholder='Enter file path to segmented volume',
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    file_format_dropdown = widgets.Dropdown(
        options=['Auto-detect', 'DICOM', 'TIFF', 'RAW', 'NIfTI', 'NumPy'],
        value='Auto-detect',
        description='Format:',
        style={'description_width': 'initial'}
    )
    
    voxel_size_x = widgets.FloatText(value=0.1, description='Voxel X (mm):', style={'description_width': 'initial'})
    voxel_size_y = widgets.FloatText(value=0.1, description='Voxel Y (mm):', style={'description_width': 'initial'})
    voxel_size_z = widgets.FloatText(value=0.1, description='Voxel Z (mm):', style={'description_width': 'initial'})
    
    load_button = widgets.Button(
        description='📂 Load Volume',
        button_style='primary',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    volume_info_display = widgets.HTML(
        value="<p><i>No volume loaded</i></p>",
        layout=widgets.Layout(height='100px', overflow='auto')
    )
    
    # ============================================
    # Section 2: Analysis Parameters
    # ============================================
    
    printing_direction = widgets.Dropdown(
        options=['X', 'Y', 'Z'],
        value='Z',
        description='Printing Direction:',
        style={'description_width': 'initial'}
    )
    
    flow_direction = widgets.Dropdown(
        options=['X', 'Y', 'Z'],
        value='Z',
        description='Flow Direction:',
        style={'description_width': 'initial'}
    )
    
    # ============================================
    # Section 3: Filament Analysis
    # ============================================
    
    filament_method = widgets.Dropdown(
        options=['distance_transform', 'cross_section'],
        value='distance_transform',
        description='Method:',
        style={'description_width': 'initial'}
    )
    
    analyze_filament_button = widgets.Button(
        description='🔬 Analyze Filaments',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    filament_results_display = widgets.HTML(
        value="<p><i>No filament analysis</i></p>",
        layout=widgets.Layout(height='150px', overflow='auto')
    )
    
    filament_visualization = Output(layout=widgets.Layout(height='300px'))
    
    # ============================================
    # Section 4: Channel Analysis
    # ============================================
    
    channel_method = widgets.Dropdown(
        options=['distance_transform', 'cross_section'],
        value='distance_transform',
        description='Method:',
        style={'description_width': 'initial'}
    )
    
    analyze_channel_button = widgets.Button(
        description='🌊 Analyze Channels',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    channel_results_display = widgets.HTML(
        value="<p><i>No channel analysis</i></p>",
        layout=widgets.Layout(height='150px', overflow='auto')
    )
    
    channel_visualization = Output(layout=widgets.Layout(height='300px'))
    
    # ============================================
    # Section 5: Porosity Analysis
    # ============================================
    
    fit_distributions_check = widgets.Checkbox(
        value=True,
        description='Fit distributions to pore sizes',
        indent=False
    )
    
    analyze_porosity_button = widgets.Button(
        description='🕳️ Analyze Porosity',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    porosity_results_display = widgets.HTML(
        value="<p><i>No porosity analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    porosity_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 6: Slice Analysis
    # ============================================
    
    n_slices = widgets.IntText(
        value=20,
        description='Number of Slices:',
        style={'description_width': 'initial'}
    )
    
    analyze_slices_button = widgets.Button(
        description='📊 Analyze Slices',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    slice_results_display = widgets.HTML(
        value="<p><i>No slice analysis</i></p>",
        layout=widgets.Layout(height='150px', overflow='auto')
    )
    
    slice_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 7: Statistical Fitting
    # ============================================
    
    variable_to_fit = widgets.Dropdown(
        options=['Select variable...'],
        value='Select variable...',
        description='Variable:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    distribution_type = widgets.Dropdown(
        options=['Gaussian', 'Poisson', 'Compare All'],
        value='Gaussian',
        description='Distribution:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    fit_button = widgets.Button(
        description='📈 Fit Distribution',
        button_style='warning',
        layout=widgets.Layout(width='150px'),
        disabled=True
    )
    
    fitting_results_output = Output(layout=widgets.Layout(height='500px', overflow='auto'))
    
    # ============================================
    # Progress and Status
    # ============================================
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        style={'bar_color': '#2ecc71'},
        layout=widgets.Layout(width='400px')
    )
    
    status_display = widgets.HTML(
        value="<p>Ready</p>",
        layout=widgets.Layout(height='60px', overflow='auto')
    )
    
    print("✅ Widgets created successfully!")


🎨 Creating interactive widgets...
✅ Widgets created successfully!


In [4]:
if WIDGETS_AVAILABLE:
    
    def load_volume_callback(button):
        """Load segmented volume"""
        global analyzer, current_volume
        
        file_path = file_path_text.value.strip()
        if not file_path:
            status_display.value = "<p style='color: red;'>Please enter a file path</p>"
            return
        
        file_path_obj = Path(file_path)
        if not file_path_obj.exists():
            data_path = project_root / 'data' / file_path
            if data_path.exists():
                file_path_obj = data_path
            else:
                status_display.value = f"<p style='color: red;'>File not found: {file_path}</p>"
                return
        
        status_display.value = "<p>Loading volume...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            analyzer = XCTAnalyzer(voxel_size=voxel_size, target_unit='mm')
            progress_bar.value = 40
            
            analyzer.load_volume(str(file_path_obj), normalize=True)
            current_volume = analyzer.volume
            progress_bar.value = 80
            
            info_html = f"""
            <h4>Volume Information</h4>
            <p><b>Shape:</b> {analyzer.volume.shape}</p>
            <p><b>Voxel Size:</b> {voxel_size} mm</p>
            <p><b>Volume Size:</b> {analyzer.volume.nbytes / (1024**2):.2f} MB</p>
            """
            volume_info_display.value = info_html
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Volume loaded successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error loading volume: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_filament_callback(button):
        """Analyze filament diameters"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing filaments...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            direction = printing_direction.value.lower()
            
            result = estimate_filament_diameter(
                current_volume,
                voxel_size,
                direction=direction,
                method=filament_method.value
            )
            analysis_results['filament'] = result
            progress_bar.value = 80
            
            # Display results
            html = f"""
            <h4>🔬 Filament Diameter Analysis</h4>
            <p><b>Method:</b> {filament_method.value}</p>
            <p><b>Direction:</b> {printing_direction.value}</p>
            <p><b>Mean Diameter:</b> {result['mean_diameter']:.4f} mm</p>
            <p><b>Std Diameter:</b> {result['std_diameter']:.4f} mm</p>
            <p><b>Min Diameter:</b> {result['min_diameter']:.4f} mm</p>
            <p><b>Max Diameter:</b> {result['max_diameter']:.4f} mm</p>
            <p><b>Number of Measurements:</b> {result.get('n_samples', len(result.get('diameters', [])))}</p>
            """
            filament_results_display.value = html
            
            # Visualize
            with filament_visualization:
                clear_output()
                if 'diameters' in result and len(result['diameters']) > 0:
                    diameters = np.array(result['diameters'])
                    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                    
                    # Histogram
                    axes[0].hist(diameters, bins=min(30, len(diameters)//2), edgecolor='black', alpha=0.7)
                    axes[0].axvline(result['mean_diameter'], color='red', linestyle='--', linewidth=2, label=f"Mean: {result['mean_diameter']:.4f} mm")
                    axes[0].set_xlabel('Diameter (mm)', fontsize=11)
                    axes[0].set_ylabel('Frequency', fontsize=11)
                    axes[0].set_title('Filament Diameter Distribution', fontsize=12, fontweight='bold')
                    axes[0].legend()
                    axes[0].grid(True, alpha=0.3)
                    
                    # Box plot
                    axes[1].boxplot(diameters, vert=True)
                    axes[1].set_ylabel('Diameter (mm)', fontsize=11)
                    axes[1].set_title('Filament Diameter Statistics', fontsize=12, fontweight='bold')
                    axes[1].grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Filament analysis complete!</p>"
            update_variable_options()
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error analyzing filaments: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_channel_callback(button):
        """Analyze channel widths"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing channels...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            direction = flow_direction.value.lower()
            
            result = estimate_channel_width(
                current_volume,
                voxel_size,
                direction=direction,
                method=channel_method.value
            )
            analysis_results['channel'] = result
            progress_bar.value = 80
            
            # Display results
            html = f"""
            <h4>🌊 Channel Width Analysis</h4>
            <p><b>Method:</b> {channel_method.value}</p>
            <p><b>Direction:</b> {flow_direction.value}</p>
            <p><b>Mean Width:</b> {result['mean_diameter']:.4f} mm</p>
            <p><b>Std Width:</b> {result['std_diameter']:.4f} mm</p>
            <p><b>Min Width:</b> {result['min_diameter']:.4f} mm</p>
            <p><b>Max Width:</b> {result['max_diameter']:.4f} mm</p>
            <p><b>Number of Measurements:</b> {result.get('n_samples', len(result.get('diameters', [])))}</p>
            """
            channel_results_display.value = html
            
            # Visualize
            with channel_visualization:
                clear_output()
                if 'diameters' in result and len(result['diameters']) > 0:
                    widths = np.array(result['diameters'])
                    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                    
                    # Histogram
                    axes[0].hist(widths, bins=min(30, len(widths)//2), edgecolor='black', alpha=0.7, color='lightblue')
                    axes[0].axvline(result['mean_diameter'], color='red', linestyle='--', linewidth=2, label=f"Mean: {result['mean_diameter']:.4f} mm")
                    axes[0].set_xlabel('Width (mm)', fontsize=11)
                    axes[0].set_ylabel('Frequency', fontsize=11)
                    axes[0].set_title('Channel Width Distribution', fontsize=12, fontweight='bold')
                    axes[0].legend()
                    axes[0].grid(True, alpha=0.3)
                    
                    # Box plot
                    axes[1].boxplot(widths, vert=True)
                    axes[1].set_ylabel('Width (mm)', fontsize=11)
                    axes[1].set_title('Channel Width Statistics', fontsize=12, fontweight='bold')
                    axes[1].grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Channel analysis complete!</p>"
            update_variable_options()
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error analyzing channels: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_porosity_callback(button):
        """Analyze porosity distribution"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing porosity...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            direction = printing_direction.value.lower()
            
            result = analyze_porosity_distribution(
                current_volume,
                voxel_size,
                printing_direction=direction,
                fit_distributions=fit_distributions_check.value
            )
            analysis_results['porosity'] = result
            progress_bar.value = 80
            
            # Display results
            profile = result['porosity_profile']
            pore_dist = result['pore_size_distribution']
            
            html = f"""
            <h4>🕳️ Porosity Analysis</h4>
            <p><b>Printing Direction:</b> {printing_direction.value}</p>
            <p><b>Mean Porosity:</b> {profile['mean_porosity']:.2%}</p>
            <p><b>Std Porosity:</b> {profile['std_porosity']:.2%}</p>
            <p><b>Min Porosity:</b> {profile['min_porosity']:.2%}</p>
            <p><b>Max Porosity:</b> {profile['max_porosity']:.2%}</p>
            <p><b>Number of Pores:</b> {pore_dist.get('n_pores', 0)}</p>
            <p><b>Mean Pore Volume:</b> {pore_dist.get('mean_pore_volume', 0):.6f} mm³</p>
            """
            porosity_results_display.value = html
            
            # Visualize
            with porosity_visualization:
                clear_output()
                fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                
                # Porosity profile
                axes[0, 0].plot(profile['positions'], profile['porosity'], 'o-', linewidth=2, markersize=4)
                axes[0, 0].axhline(profile['mean_porosity'], color='red', linestyle='--', label=f"Mean: {profile['mean_porosity']:.2%}")
                axes[0, 0].set_xlabel('Position', fontsize=11)
                axes[0, 0].set_ylabel('Porosity', fontsize=11)
                axes[0, 0].set_title('Porosity Profile Along Printing Direction', fontsize=12, fontweight='bold')
                axes[0, 0].legend()
                axes[0, 0].grid(True, alpha=0.3)
                
                # Pore size distribution
                if 'pore_volumes' in pore_dist and len(pore_dist['pore_volumes']) > 0:
                    pore_volumes = np.array(pore_dist['pore_volumes'])
                    axes[0, 1].hist(pore_volumes, bins=min(30, len(pore_volumes)//2), edgecolor='black', alpha=0.7)
                    axes[0, 1].set_xlabel('Pore Volume (mm³)', fontsize=11)
                    axes[0, 1].set_ylabel('Frequency', fontsize=11)
                    axes[0, 1].set_title('Pore Size Distribution', fontsize=12, fontweight='bold')
                    axes[0, 1].grid(True, alpha=0.3)
                
                # Porosity histogram
                axes[1, 0].hist(profile['porosity'], bins=20, edgecolor='black', alpha=0.7, color='lightgreen')
                axes[1, 0].axvline(profile['mean_porosity'], color='red', linestyle='--', linewidth=2, label=f"Mean: {profile['mean_porosity']:.2%}")
                axes[1, 0].set_xlabel('Porosity', fontsize=11)
                axes[1, 0].set_ylabel('Frequency', fontsize=11)
                axes[1, 0].set_title('Porosity Distribution', fontsize=12, fontweight='bold')
                axes[1, 0].legend()
                axes[1, 0].grid(True, alpha=0.3)
                
                # Connectivity (if available)
                if 'connectivity' in result:
                    conn = result['connectivity']
                    axes[1, 1].text(0.5, 0.5, f"Connectivity Analysis\n\n"
                                              f"Connected: {conn.get('is_connected', 'N/A')}\n"
                                              f"Number of Components: {conn.get('n_components', 'N/A')}\n"
                                              f"Largest Component: {conn.get('largest_component_size', 'N/A')}",
                                    ha='center', va='center', fontsize=12,
                                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                                    transform=axes[1, 1].transAxes)
                    axes[1, 1].axis('off')
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Porosity analysis complete!</p>"
            update_variable_options()
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error analyzing porosity: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_slices_callback(button):
        """Analyze slices along flow direction"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing slices...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            direction = flow_direction.value.lower()
            
            result = analyze_slice_along_flow(
                current_volume,
                flow_direction=direction,
                n_slices=n_slices.value,
                voxel_size=voxel_size
            )
            analysis_results['slices'] = result
            progress_bar.value = 80
            
            # Display results
            slice_metrics = result['slice_metrics']
            if len(slice_metrics) > 0:
                mean_material = np.mean([m['material_fraction'] for m in slice_metrics])
                mean_void = np.mean([m['void_fraction'] for m in slice_metrics])
                
                html = f"""
                <h4>📊 Slice Analysis</h4>
                <p><b>Flow Direction:</b> {flow_direction.value}</p>
                <p><b>Number of Slices:</b> {len(slice_metrics)}</p>
                <p><b>Mean Material Fraction:</b> {mean_material:.2%}</p>
                <p><b>Mean Void Fraction:</b> {mean_void:.2%}</p>
                """
                slice_results_display.value = html
                
                # Visualize
                with slice_visualization:
                    clear_output()
                    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                    
                    positions = [m['position'] for m in slice_metrics]
                    material_fracs = [m['material_fraction'] for m in slice_metrics]
                    void_fracs = [m['void_fraction'] for m in slice_metrics]
                    
                    # Material and void fractions
                    axes[0].plot(positions, material_fracs, 'o-', label='Material', linewidth=2, markersize=4)
                    axes[0].plot(positions, void_fracs, 's-', label='Void', linewidth=2, markersize=4)
                    axes[0].set_xlabel('Slice Position', fontsize=11)
                    axes[0].set_ylabel('Fraction', fontsize=11)
                    axes[0].set_title('Material and Void Fractions Along Flow', fontsize=12, fontweight='bold')
                    axes[0].legend()
                    axes[0].grid(True, alpha=0.3)
                    
                    # Number of regions
                    if 'n_material_regions' in slice_metrics[0]:
                        n_material = [m['n_material_regions'] for m in slice_metrics]
                        n_void = [m['n_void_regions'] for m in slice_metrics]
                        axes[1].plot(positions, n_material, 'o-', label='Material Regions', linewidth=2, markersize=4)
                        axes[1].plot(positions, n_void, 's-', label='Void Regions', linewidth=2, markersize=4)
                        axes[1].set_xlabel('Slice Position', fontsize=11)
                        axes[1].set_ylabel('Number of Regions', fontsize=11)
                        axes[1].set_title('Region Count Along Flow', fontsize=12, fontweight='bold')
                        axes[1].legend()
                        axes[1].grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Slice analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error analyzing slices: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def update_variable_options():
        """Update available variables for fitting"""
        options = ['Select variable...']
        
        if 'filament' in analysis_results:
            if 'diameters' in analysis_results['filament']:
                options.append('Filament Diameters')
        
        if 'channel' in analysis_results:
            if 'diameters' in analysis_results['channel']:
                options.append('Channel Widths')
        
        if 'porosity' in analysis_results:
            pore_dist = analysis_results['porosity'].get('pore_size_distribution', {})
            if 'pore_volumes' in pore_dist:
                options.append('Pore Volumes')
            if 'equivalent_diameters' in pore_dist:
                options.append('Pore Equivalent Diameters')
            
            profile = analysis_results['porosity'].get('porosity_profile', {})
            if 'porosity' in profile:
                options.append('Porosity Profile')
        
        variable_to_fit.options = options
        if len(options) > 1:
            variable_to_fit.disabled = False
            distribution_type.disabled = False
            fit_button.disabled = False
    
    def fit_distribution_callback(button):
        """Fit distribution to selected variable"""
        global analysis_results
        
        if variable_to_fit.value == 'Select variable...':
            status_display.value = "<p style='color: red;'>Please select a variable</p>"
            return
        
        status_display.value = "<p>Fitting distribution...</p>"
        progress_bar.value = 20
        
        try:
            # Extract data
            data = None
            var_name = variable_to_fit.value
            
            if var_name == 'Filament Diameters':
                data = np.array(analysis_results['filament']['diameters'])
            elif var_name == 'Channel Widths':
                data = np.array(analysis_results['channel']['diameters'])
            elif var_name == 'Pore Volumes':
                data = np.array(analysis_results['porosity']['pore_size_distribution']['pore_volumes'])
            elif var_name == 'Pore Equivalent Diameters':
                data = np.array(analysis_results['porosity']['pore_size_distribution']['equivalent_diameters'])
            elif var_name == 'Porosity Profile':
                data = np.array(analysis_results['porosity']['porosity_profile']['porosity'])
            
            if data is None or len(data) == 0:
                status_display.value = "<p style='color: red;'>No data available</p>"
                progress_bar.value = 0
                return
            
            data_clean = data[np.isfinite(data)]
            if len(data_clean) == 0:
                status_display.value = "<p style='color: red;'>No valid data points</p>"
                progress_bar.value = 0
                return
            
            progress_bar.value = 50
            
            # Perform fitting
            if distribution_type.value == 'Gaussian':
                fit_result = fit_gaussian(data_clean)
            elif distribution_type.value == 'Poisson':
                fit_result = fit_poisson(data_clean)
            elif distribution_type.value == 'Compare All':
                fit_result = compare_fits(data_clean, distributions=['gaussian', 'poisson'])
            
            progress_bar.value = 80
            
            # Display results
            with fitting_results_output:
                clear_output()
                print(f"📈 Distribution Fit: {var_name}")
                print("=" * 80)
                
                if distribution_type.value == 'Compare All' and isinstance(fit_result, dict) and 'best_fit' in fit_result:
                    print(f"\n✅ Best Fit: {fit_result.get('best_distribution', 'N/A')}")
                    best = fit_result['best_fit']
                    print(f"\nBest Fit Parameters:")
                    for key, value in best.items():
                        if key not in ['fitted', 'n_samples', 'distribution'] and isinstance(value, (int, float)):
                            print(f"  {key}: {value:.6f}" if isinstance(value, float) else f"  {key}: {value}")
                    
                    # Plot
                    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                    axes[0].hist(data_clean, bins=min(50, len(data_clean)//5), density=True, alpha=0.7, edgecolor='black')
                    x_range = np.linspace(data_clean.min(), data_clean.max(), 200)
                    if 'fits' in fit_result:
                        for dist_name, dist_fit in fit_result['fits'].items():
                            if dist_fit.get('fitted', False) and dist_name == 'gaussian':
                                from scipy import stats as scipy_stats
                                y = scipy_stats.norm.pdf(x_range, dist_fit['mean'], dist_fit['std'])
                                axes[0].plot(x_range, y, label=f"Gaussian (R²={dist_fit.get('r_squared', 0):.3f})", linewidth=2)
                    axes[0].set_xlabel(var_name)
                    axes[0].set_ylabel('Density')
                    axes[0].set_title(f'Distribution Fits: {var_name}')
                    axes[0].legend()
                    axes[0].grid(True, alpha=0.3)
                    
                    from scipy import stats as scipy_stats
                    scipy_stats.probplot(data_clean, dist='norm', plot=axes[1])
                    axes[1].set_title('Q-Q Plot')
                    axes[1].grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                else:
                    print(f"\n✅ Fit Parameters:")
                    for key, value in fit_result.items():
                        if key not in ['fitted', 'n_samples', 'distribution', 'params'] and isinstance(value, (int, float)):
                            print(f"  {key}: {value:.6f}" if isinstance(value, float) else f"  {key}: {value}")
                    
                    quality = evaluate_fit_quality(fit_result, data_clean)
                    print(f"\n📊 Fit Quality: {quality.get('interpretation', 'N/A')}")
                    
                    # Plot
                    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                    axes[0].hist(data_clean, bins=min(50, len(data_clean)//5), density=True, alpha=0.7, edgecolor='black')
                    x_range = np.linspace(data_clean.min(), data_clean.max(), 200)
                    if fit_result.get('distribution') == 'gaussian':
                        from scipy import stats as scipy_stats
                        y = scipy_stats.norm.pdf(x_range, fit_result['mean'], fit_result['std'])
                        axes[0].plot(x_range, y, 'r-', linewidth=2, label=f"Gaussian (R²={fit_result.get('r_squared', 0):.3f})")
                    axes[0].set_xlabel(var_name)
                    axes[0].set_ylabel('Density')
                    axes[0].set_title(f'{distribution_type.value} Fit: {var_name}')
                    axes[0].legend()
                    axes[0].grid(True, alpha=0.3)
                    
                    from scipy import stats as scipy_stats
                    scipy_stats.probplot(data_clean, dist='norm', plot=axes[1])
                    axes[1].set_title('Q-Q Plot')
                    axes[1].grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = f"<p style='color: green;'>✅ Distribution fit complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error fitting distribution: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    # Attach callbacks
    load_button.on_click(load_volume_callback)
    analyze_filament_button.on_click(analyze_filament_callback)
    analyze_channel_button.on_click(analyze_channel_callback)
    analyze_porosity_button.on_click(analyze_porosity_callback)
    analyze_slices_button.on_click(analyze_slices_callback)
    fit_button.on_click(fit_distribution_callback)
    
    print("✅ Callback functions attached!")


✅ Callback functions attached!


## 5. Display Interactive Dashboard


In [5]:
if WIDGETS_AVAILABLE:
    
    # Create loading panel
    loading_panel = widgets.VBox([
        widgets.HTML("<h2>📂 Load Segmented Volume</h2>"),
        HBox([
            file_path_text,
            file_format_dropdown
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        HBox([load_button, volume_info_display])
    ])
    
    # Create analysis parameters panel
    params_panel = widgets.VBox([
        widgets.HTML("<h3>⚙️ Analysis Parameters</h3>"),
        HBox([
            printing_direction,
            flow_direction
        ])
    ])
    
    # Create filament analysis panel
    filament_panel = widgets.VBox([
        widgets.HTML("<h3>🔬 Filament Analysis</h3>"),
        filament_method,
        analyze_filament_button,
        filament_results_display,
        filament_visualization
    ])
    
    # Create channel analysis panel
    channel_panel = widgets.VBox([
        widgets.HTML("<h3>🌊 Channel Analysis</h3>"),
        channel_method,
        analyze_channel_button,
        channel_results_display,
        channel_visualization
    ])
    
    # Create porosity analysis panel
    porosity_panel = widgets.VBox([
        widgets.HTML("<h3>🕳️ Porosity Analysis</h3>"),
        fit_distributions_check,
        analyze_porosity_button,
        porosity_results_display,
        porosity_visualization
    ])
    
    # Create slice analysis panel
    slice_panel = widgets.VBox([
        widgets.HTML("<h3>📊 Slice Analysis</h3>"),
        n_slices,
        analyze_slices_button,
        slice_results_display,
        slice_visualization
    ])
    
    # Create statistical fitting panel
    fitting_panel = widgets.VBox([
        widgets.HTML("<h3>📈 Statistical Fitting</h3>"),
        variable_to_fit,
        distribution_type,
        fit_button,
        fitting_results_output
    ])
    
    # Create tabs for organized display
    analysis_tabs = Tab(children=[
        params_panel,
        filament_panel,
        channel_panel,
        porosity_panel,
        slice_panel,
        fitting_panel
    ])
    analysis_tabs.set_title(0, '⚙️ Parameters')
    analysis_tabs.set_title(1, '🔬 Filaments')
    analysis_tabs.set_title(2, '🌊 Channels')
    analysis_tabs.set_title(3, '🕳️ Porosity')
    analysis_tabs.set_title(4, '📊 Slices')
    analysis_tabs.set_title(5, '📈 Statistics')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>🔬 Core Analysis - Morphology and Porosity</h1>"),
        loading_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📊 Analysis</h2>"),
        analysis_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start analyzing morphology and porosity.")
    print("\n💡 Tips:")
    print("   1. Load a segmented volume")
    print("   2. Set printing and flow directions")
    print("   3. Run filament, channel, and porosity analysis")
    print("   4. Explore slice analysis")
    print("   5. Fit distributions to understand data characteristics")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start analyzing morphology and porosity.

💡 Tips:
   1. Load a segmented volume
   2. Set printing and flow directions
   3. Run filament, channel, and porosity analysis
   4. Explore slice analysis
   5. Fit distributions to understand data characteristics


## 6. Summary

### What We Learned

1. **Filament Diameter Estimation**:
   - Distance transform method
   - Cross-section method
   - Statistical analysis of diameters

2. **Channel Width Analysis**:
   - Void space characterization
   - Flow path dimensions
   - Channel distribution

3. **Porosity Distribution**:
   - Porosity profile along printing direction
   - Pore size distribution
   - Connectivity analysis
   - Local porosity mapping

4. **Slice Analysis**:
   - Analysis along flow direction
   - Material and void fractions
   - Region counting
   - Spatial variation

5. **Statistical Fitting**:
   - Distribution fitting to morphological parameters
   - Goodness of fit evaluation
   - Distribution comparison

### Next Steps

- **Notebook 04**: Experimental Analysis - Flow, Thermal, Energy
  - Flow connectivity and tortuosity
  - Thermal resistance analysis
  - Energy conversion efficiency

- **Notebook 05**: Advanced Analysis
  - Sensitivity analysis
  - Virtual experiments
  - Process optimization

### Resources

- [Framework Documentation](../docs/README.md)
- [Core Modules](../docs/modules.md#core-modules)
- [Porosity Analysis](../docs/modules.md#coreporosity)
- [Filament Analysis](../docs/modules.md#corefilament_analysis)
